# Fully Connected Tokyo 2025 Karakuri Workshop

In [1]:
import logging

import sys
sys.path.append("..")

logger = logging.getLogger(__name__)

In [2]:
import os
from src.config import Config

config = Config(_env_file="../.env")
os.environ["OPENAI_API_KEY"] = config.openai_api_key.get_secret_value()

In [3]:
from src.chat import setup_weave

setup_weave(
    wandb_api_key=config.wandb_api_key.get_secret_value(),
    weave_project_name=config.weave_project_name,
)

weave: wandb version 0.22.3 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: Logged in as Weights & Biases user: k-obinata-trials.
weave: View Weave data at https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/weave


# チャット機能のお試し

In [4]:
from src.chat import chat

response = chat(messages=[{"role": "user", "content": "プランを変更したいです"}])
print(response)

weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/019a32a0-13cb-7a04-b185-a63e6bf6ed73


プランの変更は、管理画面の「プラン設定」からいつでも行うことができます。変更内容は即座に反映されます。

ご不明点がございましたら、具体的なご希望内容（例：アップグレード、ダウングレード、ご希望プラン名など）をお知らせください。さらに詳しくご案内いたします。


weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/019a32a0-21cb-73e6-93d4-b2f268244120
weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/019a32a0-5ef9-7931-908c-5ec9e2bbe7c1
weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/019a32a0-bead-7fae-89da-af6e3b835c37
weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/019a32a0-f88b-7d65-83ce-36941b432b75


# 精度評価

In [5]:
from src.run_evaluation import (
    load_evaluation_dataset,
    QAAccuracyScorer,
    ContextHallucinationScorer,
    CommonSenseHallucinationScorer,
    ChatBotModel,
)
from weave import Dataset, Evaluation, Table

dataset = load_evaluation_dataset(file_path="../data/eval-onboarding.csv")
print(f"Loaded {len(dataset)} evaluation examples")

qa_accuracy_scorer = QAAccuracyScorer()
context_scorer = ContextHallucinationScorer()
commonsense_scorer = CommonSenseHallucinationScorer()
model = ChatBotModel()

table = Table(dataset)
weave_dataset = Dataset(name="PreReleaseEvaluationDataset", rows=table)
evaluation = Evaluation(
    name="PreReleaseEvaluation",
    dataset=weave_dataset,
    scorers=[qa_accuracy_scorer, context_scorer, commonsense_scorer],
)

print("==== 評価開始 ====")
results = await evaluation.evaluate(model)

Loaded 10 evaluation examples
==== 評価開始 ====


weave: Evaluated 1 of 10 examples
weave: Evaluated 2 of 10 examples
weave: Evaluated 3 of 10 examples
weave: Evaluated 4 of 10 examples
weave: Evaluated 5 of 10 examples
weave: Evaluated 6 of 10 examples
weave: Evaluated 7 of 10 examples
weave: Evaluated 8 of 10 examples
weave: Evaluated 9 of 10 examples
weave: Evaluated 10 of 10 examples
weave: Evaluation summary {
weave:   "QAAccuracyScorer": {
weave:     "accuracy": {
weave:       "answer": {
weave:         "mean": 1.1
weave:       }
weave:     }
weave:   },
weave:   "ContextHallucinationScorer": {
weave:     "context_hallucination": {
weave:       "hallucination": {
weave:         "mean": 0.1
weave:       }
weave:     }
weave:   },
weave:   "CommonSenseHallucinationScorer": {
weave:     "common_sense_hallucination": {
weave:       "hallucination": {
weave:         "mean": 0.3
weave:       }
weave:     }
weave:   },
weave:   "model_latency": {
weave:     "mean": 3.0413529872894287
weave:   }
weave: }


# プロンプト改善

In [6]:
from src.prompts import SYSTEM_PROMPT
from src.prompt_improvement import improve_prompt_from_feedback


result = improve_prompt_from_feedback(SYSTEM_PROMPT, feedback_csv_path="../data/feedback-logs.csv")
print("=== 改善後のプロンプト ===")
print(result.improved_prompt)
print("=== 変更理由 ===")
print(result.change_reason)
print("=== フィードバック分析 ===")
print(result.feedback_analysis)

=== 改善後のプロンプト ===
## 基本情報
あなたはカスタマーサポート向けSaaS『カラクリCSデジタルシリーズ』のカスタマーサポート担当です。
常に丁寧で親切な対応を心掛けてください。

## 回答ルール
1. 回答の前に必ずFAQ・ドキュメントを参照してください。
2. FAQやドキュメントに情報がない場合は、
   - 追加の詳細質問や深堀りは行わず、
   - 必要に応じて「お手数ですが、詳細なご質問やご要望は問い合わせフォーム（https://example.com/inquiry）よりご連絡ください」とご案内してください。
3. サービスに直接関係しない一般的な質問（例：APIとは何か等）が来た場合は、
   - まず「本サービスのご利用に関するご質問でしょうか？」と意図を確認し、
   - サービス利用に関係がない場合は、一般的な説明は控えてください。
4. サービスに関する具体的なご相談や改善要望には、FAQやドキュメントの内容を案内した上で、ユーザーのニーズを深掘りしても構いません。
5. 期待値を不必要に高める表現や、調べれば答えられると誤解させる対応は避けてください。

## 例外対応
- FAQやドキュメントに情報がない内容については、必ず問い合わせフォームへの案内を含めてください。
- サービス非関連の一般的な質問には、サービス利用の意図確認を優先し、不要な詳細説明はしないでください。

このルールに従い、ユーザー体験を損なわないようにサポート対応を行ってください。
=== 変更理由 ===
主な問題は、FAQ未掲載時やサービス非関連の質問に対して過度な深堀りや追加質問を行い、ユーザーの期待値を不必要に高めてしまう点でした。改善後のプロンプトでは、FAQやドキュメントに情報がない場合は追加質問を控え、必ず問い合わせフォームへの案内を徹底するよう明記しました。また、サービスに直接関係しない一般的な質問には、まず意図確認を優先し、不要な詳細説明を避けるよう指示しています。これにより、ユーザーの手間や期待値の管理が適切になり、サービス品質とユーザー体験の向上が期待できます。
=== フィードバック分析 ===
{'total_logs': 4, 'negative_count': 3, 'positive_count': 1, 'prob

# 改善示唆出し

In [7]:
from weave import Dataset, Evaluation, Table

from src.run_improvement import load_questions, ChatBotModel, KnowledgeGapScorer

questions = load_questions(file_path="../data/eval-onboarding.csv")
print(f"Loaded {len(questions)} examples")

model = ChatBotModel()
scorer = KnowledgeGapScorer()

table = Table([{"question": q["question"], "id": q["id"]} for q in questions])
dataset = Dataset(name="KnowledgeImprovementEvaluationDataset", rows=table)
evaluation = Evaluation(
    name="KnowledgeImprovementEvaluation",
    dataset=dataset,
    scorers=[scorer],
)

print("==== 改善示唆出し開始 ====")
results = await evaluation.evaluate(model)

Loaded 10 examples
==== 改善示唆出し開始 ====


weave: Evaluated 1 of 10 examples
weave: Evaluated 2 of 10 examples
weave: Evaluated 3 of 10 examples
weave: Evaluated 4 of 10 examples
weave: Evaluated 5 of 10 examples
weave: Evaluated 6 of 10 examples
weave: Evaluated 7 of 10 examples
weave: Evaluated 8 of 10 examples
weave: Evaluated 9 of 10 examples
weave: Evaluated 10 of 10 examples
weave: Evaluation summary {
weave:   "KnowledgeGapScorer": {
weave:     "knowledge_gap": {
weave:       "flag": {
weave:         "mean": 0.2
weave:       },
weave:       "raw": {
weave:         "discontent": {
weave:           "mean": 0.2
weave:         },
weave:         "lack_of_knowledge": {
weave:           "mean": 0.2
weave:         },
weave:         "lack_of_understanding": {
weave:           "mean": 0.0
weave:         }
weave:       }
weave:     }
weave:   },
weave:   "model_latency": {
weave:     "mean": 4.23429844379425
weave:   }
weave: }
